# TEMAS - Turkey Earthquake Monitoring and Analysis System

### ==== Version: 0.7.1 released on 12 March 2023 by Marcus Zou ====


## Data Source

- The origin data table is embedded/nested at http://www.koeri.boun.edu.tr/sismo/2/latest-earthquakes/automatic-solutions/, 
- being updated daily per events happened in Turkey and surounding areas.

- From 16 Jan 2023 and onwards, I started to grab down the dataset daily and manually, 
- and put/merge into a csv file named "data/koeri.boun.edu.tr-lasteq-autosol_from_2023-01-16_tab.csv". 
- This is a very time consuming.


## Progress

#### v0.7.1 build 2023-03-12
* Dockerized the project into a cloud service
* Scheduled a Data Updater

#### v0.7.0 build 2023-03-11
* Failed to add Choropleth map due to lack of decent geojson file per province.
* Re-org the project files and folders prior to deployment.
  
#### v0.6.0 build 2023-03-10
* Split the jobs (Daily Job Runner + Mapper)
* Designed and Made a landing page (index.html) and other pages.

#### v0.5.0 build 2023-03-09
* Organized the all-in-one Jupyter Notebook: db-reader + scrapper + merger + mapper.

#### v0.4.1 build 2023-03-08
* Scraping the multiple pages from koeri.boun.edu.tr was successful.
* http://sc3.koeri.boun.edu.tr/eqevents/events1.html
* http://sc3.koeri.boun.edu.tr/eqevents/events2.html
* ...
* http://sc3.koeri.boun.edu.tr/eqevents/events32.html
* http://sc3.koeri.boun.edu.tr/eqevents/events33.html

#### v0.4.0 build 2023-03-07
* Scraping the first page from koeri.boun.edu.tr was successful.
* http://sc3.koeri.boun.edu.tr/eqevents/events.html

#### v0.3.0 build 2023-03-04
* Created a SQLite3 db and saved the dataframe of Historic data into it.

#### v0.2.1 build 2023-02-27
* Merged historic and real-time data into one local dataframe.

#### v0.2.0 build 2023-02-26
* Historic database added (from 16 Jan 2023).

#### v0.1.0 build 2023-02-13
* First release - current 500 datapoints only.


## Planning

Thinking about -
* to load up the historic data - Done!
* to build a local database (SQLite3?) to save what we have grabbed down so far - Done!
* to run a webScraping task to grab down the daily updates from the website mentioned above - Done!
* to merge the historic dataset and the realtime daily updates into one database - Done!
<br><br>
* Finnally implement this into .py files and schedule daily job-runner - Done!
    

## PART 0. Prep

In [ ]:
# Load up libraries
import os
import pandas as pd
import folium
from folium import plugins
import branca
import geocoder
import reverse_geocoder as rg
import webbrowser
import datetime as dt
import sqlite3
import requests
from bs4 import BeautifulSoup

print('Pandas: ', pd.__version__)
print('Folium: ', folium.__version__)

## PART 1. Web Scrapping Real-time Data


More in-depth digging reveals that the actual tables are in HTML format as below (34 counts):

* http://sc3.koeri.boun.edu.tr/eqevents/events.html
* http://sc3.koeri.boun.edu.tr/eqevents/events1.html
* http://sc3.koeri.boun.edu.tr/eqevents/events2.html
* ...
* http://sc3.koeri.boun.edu.tr/eqevents/events32.html
* http://sc3.koeri.boun.edu.tr/eqevents/events33.html

Then the target page is being reset to: http://sc3.koeri.boun.edu.tr/eqevents/events{i}.html and 
the scrapping task focus on the first a few pages if the job runner be turned on every day.

It turns out the first page naming convention **differs** from the rest, then we have to scrap them into seperate dataframes and merge the two.

Then merge the dataframe to the SQLite3 database.


In [ ]:
# send a GET request to the URL - the first page
url0 = "http://sc3.koeri.boun.edu.tr/eqevents/events.html"
response = requests.get(url0)
# parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

data0 = []
# find the table element and iterate over its rows
table = soup.find("table", {'class': 'index'})
rows = table.find_all("tr", {'class': 'trIndevnrow'})[1:]
for row in rows:
    # get the cells in the row
    cols = row.find_all("td")
    # get the earthquake information from the columns
    origintimeutc = cols[0].text.strip()
    magnitude = cols[1].text.strip()
    magType = cols[2].text.strip()
    lat = cols[3].text.strip()
    long = cols[4].text.strip()
    depth = cols[5].text.strip()
    region = cols[6].text.strip()
    am = cols[7].text.strip()
    lastUpd = cols[8].text.strip()
    kml = cols[9].text.strip()
    
    data0.append([origintimeutc, magnitude, magType, lat, long, depth, region, am, lastUpd, kml])
        
df0 = pd.DataFrame(data0, columns=['origintimeutc', 'magnitude', 'magType', 'latitude', 'longitude', 'depthKM', 'region', 'measMethod', 'updTime', 'attribute'])
df0

In [ ]:
# Scrapping Multiple Pages
data1 = []
# loop through the pages 1-5
for i in range(1,9):
    # specify the URL to scrape
    url = f"http://sc3.koeri.boun.edu.tr/eqevents/events{i}.html"
    
    # make a GET request to the URL and get the HTML content
    response = requests.get(url)
    html_content = response.content
    
    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # find the earthquake table
    eq_table = soup.find('table', {'class': 'index'})
    
    # loop through the rows of the table
    for row in eq_table.find_all('tr', {'class': 'trIndevnrow'})[1:]:
        # get the columns of the row
        cols = row.find_all('td')
        
        # get the earthquake information from the columns
        origintimeutc = cols[0].text.strip()
        magnitude = cols[1].text.strip()
        magType = cols[2].text.strip()
        lat = cols[3].text.strip()
        long = cols[4].text.strip()
        depth = cols[5].text.strip()
        region = cols[6].text.strip()
        am = cols[7].text.strip()
        lastUpd = cols[8].text.strip()
        kml = cols[9].text.strip()
        
        data1.append([origintimeutc, magnitude, magType, lat, long, depth, region, am, lastUpd, kml])
               
df1 = pd.DataFrame(data1, columns = ["origintimeutc", "magnitude", "magType", "latitude", "longitude", "depthKM", "region", "measMethod", "updTime", "attribute"])
df1


In [ ]:
# Concatenate 2 realtime dataframes
rtDF = pd.concat([df0, df1]).drop_duplicates(subset='origintimeutc', keep='last')
# Reset the Index
rtDF = rtDF.reset_index(drop=True)
rtDF

## PART 2. Load Historic Data from Local Database


In [ ]:
# Create a connection to the databse
conn = sqlite3.connect('data/earthquake-in-turkey.db')

# Read out the whole dataset as dataframe
histDF = pd.read_sql_query("SELECT * FROM quaketk", conn)
histDF

In [ ]:
# Determine the last timestamp of the Historic database
cutoff_time = histDF['origintimeutc'].max()
print('The newest timestamp in Historic database is:', cutoff_time)

# Subset the Realtime DF per the last record in the Historic Database
rtDF = rtDF[rtDF['origintimeutc'] > cutoff_time]

# Sort the DF to be the format of: Older datapoint to be inserted firstly
rtDF1 = rtDF.sort_values(by='origintimeutc', ascending=True)
rtDF1.reset_index(drop=True, inplace=True)
rtDF1


## PART 3. Merge/Insert Realtime DF into Historic Database

In [ ]:
# Save the dataframe to database
rtDF1.columns = ['origintimeutc', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute']
rtDF1.to_sql('quaketk', conn, if_exists='append', index=False)


In [ ]:
# Verify if the dataframe has been saved to sqlite DB or not, please uncomment the following command
#conn.execute('SELECT * from quaketk ORDER BY origintimeutc DESC LIMIT 20').fetchall()

In [ ]:
# Cloe the Database
conn.close

## PART 4. Retrieve the Whole Dataset for Export and Display

In [ ]:
# Setup pandas display template
pd.set_option('display.width', 960)
pd.set_option('display.max_columns', 9)

In [ ]:
# Create a connection to the databse
conn = sqlite3.connect('data/earthquake-in-turkey.db')

# Read out the whole dataset as dataframe
df_d = pd.read_sql_query("SELECT * FROM quaketk", conn)
df_d

In [ ]:
# Print the dataset range
print('Time Range of the Dataset is between:', df_d['origintimeutc'].min(), 'and', df_d['origintimeutc'].max())

print("\ndf.columns:\n", df_d.columns)

## PART 5. Data Re-treatment and Export Data Table in HTML format

In [ ]:
# Re-treatment of the dataset
df_d['origintimeutc'] = df_d['origintimeutc'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df_d['magnitude'] = df_d['magnitude'].astype('float')
df_d['magtype'] = df_d['magtype'].astype('string')

df_d['latitude'] = df_d['latitude'].astype(str).map(lambda x: x.rstrip('° N').rstrip('° S'))
df_d['longitude'] = df_d['longitude'].astype(str).map(lambda x: x.rstrip('° E').rstrip('° W'))
df_d['depthkm'] = df_d['depthkm'].replace('-', 0).astype("float")

df_d['region'] = df_d['region'].astype('string')
df_d['measmethod'] = df_d['measmethod'].astype('string')
df_d['updtime'] = df_d['updtime'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df_d['attribute'] = df_d['attribute'].astype('string')

# Adjust datetime from UTC (GMT) to Turkey timezone (GMT+3)
df_d['eventtime'] = df_d['origintimeutc'] + pd.DateOffset(hours=3)
df_d['updtime'] = df_d['updtime'] + pd.DateOffset(hours=3)
df_d['eventtime'] = pd.to_datetime(df_d['eventtime'])
df_d['updtime'] = pd.to_datetime(df_d['updtime'])


# Create new columns for date and time
df_d['date'] = pd.to_datetime(df_d['eventtime']).dt.date
df_d['time'] = pd.to_datetime(df_d['eventtime']).dt.time

# take a look
df_d.tail()


In [ ]:
# Interactive data table using Dtale - Big fun, isn't it?
import dtale
dtale.show(df_d)

In [ ]:
# Preapre Data for exporting to a Data Table

# Select columns
df_tbl = df_d[['eventtime', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute']]
df_tbl.columns = ['eventTime', 'magnitude', 'magType', 'latitude', 'longitude', 'depthKm', 'region', 'measMethod', 'updTime', 'attribute']
# Change the default Ascending to Descending to put newest datapoints on the top of the data table
df_tbl_1 = df_tbl.sort_values(by='eventTime', ascending=False)

In [ ]:
# Self-making Interactive Data Table using JQuery
def generate_html(dataframe: pd.DataFrame):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id="table")
    # construct the complete HTML with jQuery Data tables
    # You can disable paging or enable y scrolling on lines 20 and 21 respectively
    html = f"""
    <html>
    <header>
        <link href="https://cdn.datatables.net/1.11.5/css/jquery.dataTables.min.css" rel="stylesheet">
    </header>
    <body>
    {table_html}
    <script src="https://code.jquery.com/jquery-3.6.0.slim.min.js" integrity="sha256-u7e5khyithlIdTpu22PHhENmPcRdFiHRjhAuHcs05RI=" crossorigin="anonymous"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.11.5/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                order: [[0, 'desc']],
                lengthMenu: [20, 50, 100, 200],
                pageLength: 20,
                paging: true,
            }});
        }});
    </script>
    </body>
    </html>
    """
    # return the html
    return html

# Give a go
table_interact = generate_html(df_tbl_1)
with open("web/table_interact_koeri.html", 'w') as f:
    f.write(table_interact)
webbrowser.open(os.getcwd() + "../web/table_interact_koeri.html")


In [ ]:
# Another Pretty HTML Data Table
from pretty_html_table import build_table

html_table_bluelight = build_table(df_tbl_1, 'blue_light',
                                   font_size='medium',
                                   font_family='Arial',
                                   text_align='center',
                                   width='auto',
                                   index=False,
                                   conditions={
                                        'magnitude': {
                                        'min': 3.0,
                                        'max': 9.0,
                                        'mix_color': 'green',
                                        'max_color': 'red'
                                        }
                                   },
                                   even_color='black',
                                   even_bg_color='white'
                                   )

# center the table using HTML formatting
html_table_bluelight = '<div style="text-align:center">' + html_table_bluelight + '</div>'

# write the HTML table to a file
with open('web/table_bluelight_koeri.html', 'w') as f:
    f.write(html_table_bluelight)
webbrowser.open(os.getcwd() + "../web/table_bluelight_koeri.html")


## PART 6. Export Maps

### 6.1 Bubble Map

In [ ]:
# Subset the df and rename the columns
df_d2 = df_d[['date', 'time', 'latitude', 'longitude', 'depthkm', 'magnitude', 'magtype', 'region', 'measmethod', 'eventtime', 'updtime', 'attribute']]
df_d2.head()
# print the earthquake with magnitude >=4
print(df_d2[df_d2['magnitude'].astype('float') >= 5])

In [ ]:
# Define our functions
# Func 1 - Proportionize the fill_color against magnitude
myColorScheme = ['green', 'red', 'black']
def get_color(magnitude):
    if magnitude <= 3:
        return myColorScheme[0]
    elif magnitude <= 6:
        return myColorScheme[1]
    else:
        return myColorScheme[2]

# Func 2 - addCircles
popwidth, popht = 250, 150
def addCircles(df, map):
    for x, y, mag, region, date, time in zip(df['latitude'], df['longitude'], df['magnitude'], df['region'], df['date'], df['time']):
        popUp = f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Magnitude: {mag} </strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Region: {region}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Date: {date}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Time: {time}</strong></span></p>"
        iframe = folium.IFrame(popUp, width=popwidth, height=popht)
        popup = folium.Popup(iframe, max_width=450)
        folium.CircleMarker(location=(x, y), radius=float(mag) * 4, weight=2, opacity=1, popup=popup,
                            color=get_color(mag), fill_color=get_color(mag), fill_opacity=0.6).add_to(map)

In [ ]:
# Map Parameters
magThreshold = 4
mapCtr = [39.16, 35.66]

# Initial Map
bubbleMap = folium.Map(location=mapCtr, zoom_start=6, tiles=None)

# Load up the tectonic polygon (in GeoJson format)
# Original URL - https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json
folium.GeoJson('data/PB2002_boundaries.json', name="Tectonic Boundaries").add_to(bubbleMap)

# Add tiles with custom names (The first tile will be the default)
folium.TileLayer('Stamen Terrain', name='Terrian').add_to(bubbleMap)
folium.TileLayer('openstreetmap', name='Open Street').add_to(bubbleMap)
folium.TileLayer('Stamen Toner', name='Toner').add_to(bubbleMap)
folium.TileLayer('Stamen Water Color', name='Water Color').add_to(bubbleMap)
folium.TileLayer('cartodbdark_matter', name='Dark Matter').add_to(bubbleMap)
folium.LayerControl().add_to(bubbleMap)

# Map Data filtering
earthquakeDF = df_d2[df_d2['magnitude'].astype('float') >= magThreshold]

# add Circles (sizing per magnitude)
addCircles(earthquakeDF, bubbleMap)

# Add lenend to the bubble map
from branca.colormap import StepColormap
magMin, magLow, magHigh, magMax = 0, 3, 6, 10
mySteps = [magMin, magLow, magHigh, magMax]

# myColorScheme has been defined previously
legend_bar = StepColormap(colors=myColorScheme,
                          index=mySteps,
                          vmin=magMin, vmax=magMax,
                          tick_labels = mySteps)
legend_bar.caption = 'Magnitude Scale'
legend_bar.add_to(bubbleMap)

# More widgets
bubbleMap.add_child(folium.LatLngPopup())
bubbleMap

In [ ]:
# Eventually save map to interactive HTML file and display in the browser
bubbleMap.save("web/map_bubble_koeri.html")
webbrowser.open(os.getcwd() + "../web/map_bubble_koeri.html")

### 6.2 Heat Map

In [ ]:
# Heat Map
import ipywidgets as widgets
from IPython.display import display

# Make a list of heatMapData
heatMapData = earthquakeDF[['latitude', 'longitude', 'magnitude']]

# Create the HeatMap - "CartoDB Dark_Matter" is the default tile for heat map
heatMap = folium.Map(mapCtr, tiles="Stamen Terrain", zoom_start=6)
# More widgets
heatMap.add_child(folium.LatLngPopup())

# Create heat map
plugins.HeatMap(heatMapData, 
                name="Magnitude", 
                radius=earthquakeDF['magnitude'].mean()*3,
                min_opacity = 1,
                max_zoom=15,
                blur=20,
                overlay=False,
                control=False,
                show=True
                ).add_to(heatMap)

# Create a toolbar using ipywidgets in Jupyter Notebook
toolbar_left = widgets.HTML(
    value='<h3 style="text-align:left; background-color: purple; color: white">Turkey Earthquake Monitoring System - Heat Map</h3>',
    layout=widgets.Layout(width='70%', height='50px')
)
toolbar_right = widgets.HTML(
    value='<h3 style="text-align:right; background-color: light-grey; color: blue"> \
    <a href="output/map_bubble_koeri.html" target="_blank">Switch to Bubble Map</a></h3>',
    layout=widgets.Layout(width='30%', height='50px')
)

toolbar = widgets.HBox([toolbar_left, toolbar_right])

# Display the toolbar and the map together using IPython.display
display(toolbar)
display(heatMap)


In [ ]:
# Save Heatmap and display in the browser
heatMap.save("web/map_heat_koeri.html")
webbrowser.open(os.getcwd() + "../web/map_heat_koeri.html")

### 6.3 Choropleth Map

__Per Wikipedia__

A choropleth map (from Greek χῶρος (choros) 'area/region', and πλῆθος (plethos) 'multitude') is a type of statistical thematic map that uses pseudocolor, i.e., color corresponding with an aggregate summary of a geographic characteristic within spatial enumeration units, such as population density or per-capita income.

```
#####  The data sample have issue, then disable this for now
# Map Parameters
magThreshold = 4
mapCtr = [39.16, 35.66]

# Load up the turkey provinces polygon (in GeoJson format)
province_geo = pd.read_json('data/geoboundaries-TUR-ADM1_simplified.json')
province_data = pd.read_csv("data/geodata-TUR-ADM1_mag_sum_avg.csv")
#province_data.columns

merged_data = province_geo.merge(province_data, left_on='shapeISO', right_on='shapeISO')

# Initial Map
choropMap = folium.Map(location=mapCtr, zoom_start=5)

folium.Choropleth(
    geo_data=merged_data,
    name="choropleth Map of Turkey Earthquake",
    data=merged_data,
    columns=["shapeISO", "magSum"],
    key_on="feature.properties.shapeISO",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Earthquake Magnitude sum per Province",
).add_to(choropMap)

folium.LayerControl().add_to(choropMap)

choropMap
```

In [ ]:
# Showcase of Reverse Geocoding
import reverse_geocoder as rg
g = rg.search(mapCtr)
print(g) # g is a list with one dictionary
print('City:', g[0]['name'], '| Province:', g[0]['admin1'], '| Country:', g[0]['cc'])


##### Function 3 - Reverse Geocoding - Takes 51 minutes, then comments out for this moment
```
import reverse_geocoder as rg
def get_location(row):
    coordinates = (row['latitude'], row['longitude'])
    result = rg.search(coordinates)[0]
    return result['name'], result['admin1'], result['cc']

df2[['city', 'state', 'country']] = df2.apply(get_location, axis=1, result_type='expand')

df2
```

## 6.4 Time-Slider Choropleth Map

Folium is a Python library for creating interactive maps. It allows you to create maps with different types of markers, shapes, and layers. Folium also provides a way to add a time slider to your maps, which allows you to display data over time.

To add a time slider to your Folium map, you can use the plugins.TimestampedGeoJson class. This class takes a GeoJSON object as input, along with a date field that indicates the time for each feature in the GeoJSON object. Here's an example of how to use it:

In [ ]:
# Map Data filtering
df_d = df_d[df_d['magnitude'].astype('float') >= 4]
df_d.tail()

In [ ]:
import pandas as pd
import geopandas as gpd

# Create a sample DataFrame with latitude, longitude, and time columns
geo_data = df_d[['latitude', 'longitude', 'magnitude', 'depthkm', 'region', 'date']]
geo_data.reset_index(drop=True, inplace=True)
geo_data.tail()

In [ ]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(geo_data, geometry=gpd.points_from_xy(geo_data.longitude, geo_data.latitude))
gdf.crs = 'EPSG:4326'

gdf

In [ ]:
import folium
from folium.plugins import TimeSliderChoropleth
import pandas as pd

# Create map object
mapCtr = [39.16, 35.66]

m = folium.Map(location=mapCtr, zoom_start=6)

gdf['date'] = pd.to_datetime(gdf['date'], format = '%Y-%m-%d')

gdf['date'] = pd.to_datetime(gdf['date'], errors='coerce')
gdf = gdf.dropna(subset=['date', 'geometry'])

gdf

In [ ]:
print(gdf.columns)
gdf = gdf.set_index('date')
print(gdf.columns)

In [ ]:
import datetime
style_dict = {'fillColor': {'property': 'my_property', 'scale': 'YlOrRd', 'domain': [0, 1]},
              'fillOpacity': {'value': 0.5},
              'weight': {'value': 0}}

g = TimeSliderChoropleth(
    gdf,
    styledict=style_dict,
).add_to(m)

m


## PART 7. Bootstrap the Landing page and Other pages

This has been done by bootstraping a very simple landing page, consisting of:
* a Toolbar with
    * Home
    * Data Table
    * Bubble Map
    * Heat Map
    * Project
* a Map container, where
    * the content shall be switched over from one to another once visitor clicks the links on the toolbar;
    * a default page shall be loaded one the website is open.

So far, all look good.

## The END